In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import word_tokenize, sent_tokenize
from bs4 import BeautifulSoup
import re
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings

warnings.filterwarnings('ignore')

# Download NLTK data
nltk.download('stopwords')
nltk.download('punkt')

# Importing the training data
imdb_data = pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')
print(imdb_data.shape)
print(imdb_data.head(10))

# Summary of the dataset
print(imdb_data.describe())

# Sentiment count
print(imdb_data['sentiment'].value_counts())

# Split the dataset
# Train dataset
train_reviews = imdb_data.review[:40000]
train_sentiments = imdb_data.sentiment[:40000]
# Test dataset
test_reviews = imdb_data.review[40000:]
test_sentiments = imdb_data.sentiment[40000:]
print(train_reviews.shape, train_sentiments.shape)
print(test_reviews.shape, test_sentiments.shape)

tokenizer = ToktokTokenizer()
# Setting English stopwords
stopword_list = nltk.corpus.stopwords.words('english')

# Removing the HTML strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub(r'\[[^]]*\]', '', text)

# Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

# Apply function on review column
imdb_data['review'] = imdb_data['review'].apply(denoise_text)

# Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-Z0-9\s]'
    text = re.sub(pattern, '', text)
    return text

# Apply function on review column
imdb_data['review'] = imdb_data['review'].apply(remove_special_characters)

# Stemming the text
def simple_stemmer(text):
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

# Apply function on review column
imdb_data['review'] = imdb_data['review'].apply(simple_stemmer)

# Removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

# Apply function on review column
imdb_data['review'] = imdb_data['review'].apply(remove_stopwords)

# Normalized train reviews
norm_train_reviews = imdb_data.review[:40000]
# Normalized test reviews
norm_test_reviews = imdb_data.review[40000:]

# Count vectorizer for bag of words
cv = CountVectorizer(min_df=0, max_df=1, binary=False, ngram_range=(1, 3))
# Transformed train reviews
cv_train_reviews = cv.fit_transform(norm_train_reviews)
# Transformed test reviews
cv_test_reviews = cv.transform(norm_test_reviews)

print('BOW_cv_train:', cv_train_reviews.shape)
print('BOW_cv_test:', cv_test_reviews.shape)

# Tfidf vectorizer
tv = TfidfVectorizer(min_df=0, max_df=1, use_idf=True, ngram_range=(1, 3))
# Transformed train reviews
tv_train_reviews = tv.fit_transform(norm_train_reviews)
# Transformed test reviews
tv_test_reviews = tv.transform(norm_test_reviews)

print('Tfidf_train:', tv_train_reviews.shape)
print('Tfidf_test:', tv_test_reviews.shape)

# Labeling the sentiment data
lb = LabelBinarizer()
# Transformed sentiment data
sentiment_data = lb.fit_transform(imdb_data['sentiment'])
print(sentiment_data.shape)

# Splitting the sentiment data
train_sentiments = sentiment_data[:40000]
test_sentiments = sentiment_data[40000:]
print(train_sentiments)
print(test_sentiments)

# Training the model
lr = LogisticRegression(penalty='l2', max_iter=500, C=1, random_state=42)
# Fitting the model for Bag of Words
lr_bow = lr.fit(cv_train_reviews, train_sentiments)
print(lr_bow)
# Fitting the model for TFIDF features
lr_tfidf = lr.fit(tv_train_reviews, train_sentiments)
print(lr_tfidf)

# Predicting the model for Bag of Words
lr_bow_predict = lr.predict(cv_test_reviews)
print(lr_bow_predict)
# Predicting the model for TFIDF features
lr_tfidf_predict = lr.predict(tv_test_reviews)
print(lr_tfidf_predict)

# Accuracy score for Bag of Words
lr_bow_score = accuracy_score(test_sentiments, lr_bow_predict)
print("lr_bow_score:", lr_bow_score)
# Accuracy score for TFIDF features
lr_tfidf_score = accuracy_score(test_sentiments, lr_tfidf_predict)
print("lr_tfidf_score:", lr_tfidf_score)

# Classification report for Bag of Words
lr_bow_report = classification_report(test_sentiments, lr_bow_predict, target_names=['Positive', 'Negative'])
print(lr_bow_report)

# Classification report for TFIDF features
lr_tfidf_report = classification_report(test_sentiments, lr_tfidf_predict, target_names=['Positive', 'Negative'])
print(lr_tfidf_report)

# Confusion matrix for Bag of Words
cm_bow = confusion_matrix(test_sentiments, lr_bow_predict, labels=[1, 0])
print(cm_bow)
# Confusion matrix for TFIDF features
cm_tfidf = confusion_matrix(test_sentiments, lr_tfidf_predict, labels=[1, 0])
print(cm_tfidf)

# Training the linear SVM
svm = SGDClassifier(loss='hinge', max_iter=500, random_state=42)
# Fitting the SVM for Bag of Words
svm_bow = svm.fit(cv_train_reviews, train_sentiments)
print(svm_bow)
# Fitting the SVM for TFIDF features
svm_tfidf = svm.fit(tv_train_reviews, train_sentiments)
print(svm_tfidf)

# Predicting the model for Bag of Words
svm_bow_predict = svm.predict(cv_test_reviews)
print(svm_bow_predict)
# Predicting the model for TFIDF features
svm_tfidf_predict = svm.predict(tv_test_reviews)
print(svm_tfidf_predict)

# Accuracy score for Bag of Words
svm_bow_score = accuracy_score(test_sentiments, svm_bow_predict)
print("svm_bow_score:", svm_bow_score)
# Accuracy score for TFIDF features
svm_tfidf_score = accuracy_score(test_sentiments, svm_tfidf_predict)
print("svm_tfidf_score:", svm_tfidf_score)

# Classification report for Bag of Words
svm_bow_report = classification_report(test_sentiments, svm_bow_predict, target_names=['Positive', 'Negative'])
print(svm_bow_report)
# Classification report for TFIDF features
svm_tfidf_report = classification_report(test_sentiments, svm_tfidf_predict, target_names=['Positive', 'Negative'])
print(svm_tfidf_report)

# Confusion matrix for Bag of Words
cm_bow = confusion_matrix(test_sentiments, svm_bow_predict, labels=[1, 0])
print(cm_bow)
# Confusion matrix for TFIDF features
cm_tfidf = confusion_matrix(test_sentiments, svm_tfidf_predict, labels=[1, 0])
print(cm_tfidf)

# Training the Multinomial Naive Bayes
mnb = MultinomialNB()
# Fitting the model for Bag of Words
mnb_bow = mnb.fit(cv_train_reviews, train_sentiments)
print(mnb_bow)
# Fitting the model for TFIDF features
mnb_tfidf = mnb.fit(tv_train_reviews, train_sentiments)
print(mnb_tfidf)

# Predicting the model for Bag of Words
mnb_bow_predict = mnb.predict(cv_test_reviews)
print(mnb_bow_predict)
# Predicting the model for TFIDF features
mnb_tfidf_predict = mnb.predict(tv_test_reviews)
print(mnb_tfidf_predict)

# Accuracy score for Bag of Words
mnb_bow_score = accuracy_score(test_sentiments, mnb_bow_predict)
print("mnb_bow_score:", mnb_bow_score)
# Accuracy score for TFIDF features
mnb_tfidf_score = accuracy_score(test_sentiments, mnb_tfidf_predict)
print("mnb_tfidf_predict:", mnb_tfidf_predict)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


(50000, 2)
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
5  Probably my all-time favorite movie, a story o...  positive
6  I sure would like to see a resurrection of a u...  positive
7  This show was an amazing, fresh & innovative i...  negative
8  Encouraged by the positive comments about this...  negative
9  If you like original gut wrenching laughter yo...  positive
                                                   review sentiment
count                                               50000     50000
unique                                              49582         2
top     Loved today's show!!! It was a variety and not...  positive
freq                    

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
